# The Experiment Settings Class
In this section we will learn how to use OOP principles to create an experiments settings class that will hold all experiment settings and that we can instantiate in other modules. In doing so, we will use the simplified Stroop experiment that we programmed in chapter 9 and move all functionalities related to settings (e.g. settings dictionary, methods for loading from files, methods for getting and setting instructions text) into a separate **settings class**. Specifically we will cover the following topics:
- Settings Class Setup
- Modules and Classes
- Classes as Independent Entities
---

## Settings class setup

Our goal in this section is to replace the `config.py` settings logic by a settings class that we can then instantiate in our python module were the experiment logic resides. In order to achieve this, we will model all *constants* as class attributes and settings as insztance attributes that can be filled as needed by the respective class methods via **getter and setter methods**.

First we will move all imports that are related to settings to the `config.py` module:
```python
# imports
import os
from collections import OrderedDict
from datetime import datetime
import csv
import random
from itertools import zip_longest
import pygame
```

Next, we create a class called `Settings()` and define the constants as class attributes. Note that the results dictionary is also a class attribute:

```python
class Settings():

    # class attributes
    bgColor = (180, 180, 180) # bg is light grey
    blackColor = (0, 0, 0) # text is black
    redColor = (250, 0, 0) # red color
    blueColor = (0, 0, 250) # blue color
    screenSize = (1200, 800) # set screen size
    lineLength = 40 # line length of fixcross
    lineWidth = 5 # line width of fixcross
    FPS = 60 # frames per second

    # results dicts
    results = OrderedDict([("id", []),
                           ("age", []),
                           ("gender", []),
                           ("major", []),
                           ("items", None),
                           ("colors", None),
                           ("groundtruth", []),
                           ("responses", []),
                           ("rts", []),
                                   ])
```

In the constructor, defined by the `__init__(self)` method, we are placing all settings and fill them with the respective class methods:
```python
    # instance attributes (constructor)
    def __init__(self):

        # init experiment and pygame
        self.init_pygame()
        self.init_experiment()

        # variable instance placeholders
        self.verPoints = None # placeholder for vert. points of fixcross
        self.horPoints = None # placeholder for hor. points of fixcross
        self.stimlist = None # placeholder for stimulus list
        self.item = None # placeholder for the item to be rendered
        self.itemRect = None # placeholder for item rectangle
        self.response = None # variable holding temporary response

        # instance attributes we fill as needed
        self.instWidth = self.screenSize[0] - self.screenSize[0] // 10 # placeholder for instruction width
        self.instHeight = self.screenSize[1] - self.screenSize[1] // 10 # placeholder for instruction height
        self.instPath = self.create_filepath("instructions") # placeholder for relative path
        self.stimuliPath = self.create_filepath("stimuli") # placeholder for stimuli path
        self.dataPath = self.create_filepath("data") # pöaceholder for data path
        self.continueVal=  0 # boolean value to control continue events
        self.starter = 0 # boolean value to control task start events
        self.quit = 0 # boolean value to control closing experiment at end
        self.filename = self.get_filename() # placeholder for filename

        # stimuli loading
        self.load_stimuli()

        # instructions placeholders
        self.inst_welcome = self.load_instructions("welcome.txt") # placeholder for welcome text
        self.inst_intro1 = self.load_instructions("intro1.txt") # placeholder for intro 1 text
        self.inst_intro2 = self.load_instructions("intro2.txt") # placeholder for intro 2 text
        self.inst_startTask = self.load_instructions("starttask.txt") # placeholder for starting task text
        self.inst_endTask = self.load_instructions("endtask.txt") # placeholder for end task text
        self.inst_goodbye = self.load_instructions("goodbye.txt") # placeholder for goodbye tex
        
```


In the constructor there are different sections which all together make up the settings instance attributes. In order for these to be available and usable once the class is instantiated and the constructor is implicitly called, there are some points we have to consider. In the following thes points are explained and broken down in detail:

- Initialization of pygame and experiment settings
- Placeholders that are empty (`None`) but will be used during experiment
- Setting of instance attributes
- Stimuli loading and setting of instructions texts


*Initialization of pygame and experiment settings*: <br>Note that in the first two lines of the constructor, we call two methods that initialize pygame, as we need it to first initialized in order to use the pygame attributes and methods (e.g. for defining shapes of rectangles). After that we initialize the experiment, where we use the pygame functionalities ti set up the experiments base settings. The implementations of these methods is as follows:
```python
    def init_pygame(self):
        """init pygame explicitly."""

        # initialize pygame modules
        pygame.init()
        pygame.mouse.set_visible(False) # disable mouse

        # set frame rate
        clock = pygame.time.Clock()
        clock.tick(self.FPS)

    def init_experiment(self):
        """
        initializes pygame backends explicitly with
        predefined settings.
        """
        # get demographics
        self.demographics_input()

        # set and innit necessary pygame features
        self.screen = pygame.display.set_mode(self.screenSize, pygame.FULLSCREEN) # placeholder for screen instance
        self.itemFont = pygame.font.SysFont("Arial", 40) # placeholder for item font
        self.instFont = pygame.font.SysFont("Arial", 30) # placeholder for instructions font
        self.screenRect = self.screen.get_rect()
```

*Placeholders*:<br> In the next lines of the constructor, a few atttributes are set to null, so they can letter be filled with values once that settings class is instantiated:
```python
        # variable instance placeholders
        self.verPoints = None # placeholder for vert. points of fixcross
        self.horPoints = None # placeholder for hor. points of fixcross
        ...
```

*Setting of instance attributes*:<br> Then, instance attributes that need to have values in it upon instantiation are set. Note that some attributes are set directly (e.g. `self.instWidth = self.screenSize[0] - self.screenSize[0] // 10`) and others are set via a function call (e.g.`self.instPath = self.create_filepath("instructions")`). In this case, filpaths are created for `instructions`, `stimuli`, and `data`. The method implementation is as follows:
```python
    def create_filepath(self, appended_text_to_abs_path):
        """ get os path and append to it custom directory."""

        absPath = os.path.abspath(os.curdir)
        finalPath = os.path.join(absPath, appended_text_to_abs_path)
        return finalPath
```

Note that when the class methods are called within the class, the `self` keyword always needs to be placed in front of it, because we are referring to the same calss instance. For the same reason `self` needs to be passed as first argument in the funtion definition.

*Stimuli loading and setting of instructions texts*:<br> In the last part, the text blocks that will be used in the experiment are created (e.g.`self.inst_welcome = self.load_instructions("welcome.txt")`). This is done by using the following method:
```python
    def load_instructions(self, filename):
        """
        loads instructions from a text file.
        arg: name of file
        returns: content of file
        """

        # open file
        with open(os.path.join(self.instPath, filename), 'r') as file:
            infile = file.read()
        # return content as string
        return infile
```

As mentioned earlier, important to note here is that most helper methods in the `Settings()` class are used to set instance attributes that can be used once the class is instantiated. However, there are also methods that will be used in the experiment logic. One example is the method for saving results:
```python
def save_results(self, filename, resultsdict):
        """
        saves results to a csv file.
        arg1: filename
        arg2: dictionary holding resultsdict
        """
        # open data file
        with open(os.path.join(self.dataPath, filename), 'w', newline="") as file:
            # create csv writer
            w = csv.writer(file, delimiter=';')
            # write first row (variable labels)
            w.writerow(resultsdict.keys())
            # write data row wise
            w.writerows(zip_longest(*resultsdict.values()))
```
This method will be used in the main logic at the end of the experiment.

## Modules and Classes

As you already know python modules can be impored into other modules via the `import` statement. Now that we have a settings class implementation in the config.py module, we can easily import the class into the main `stroop.py` module and use all of the attributes as before. The import and instantiation work like so:
```python
from config import Settings

# instatiate classes
settings = Settings()
```

From there on all the attributes and methods can be accessed via `settings.______`. This is identicall to how we use the pygame **application interface (API)**: `pygame.______`. Now it becomes clear that all libraries that we use (e.g. pygame, etc.) all have classes, methods or plain functions in them that we can use and Pygame is not exception here. Let's take a look at how the `settings` instance can be used in our code:
```python
def start_welcome_block():
    """presents welcome instructions to participant."""

    # set background
    settings.screen.fill(settings.bgColor)
    settings.continueVal = 0

    while settings.continueVal != 1:

        # create welcome instruction object
        welcomeInst = TextPresenter.text_object(settings.inst_welcome, settings.instFont,
                                                settings.instWidth, settings.instHeight)
        # blit instructions to screen
        settings.screen.blit(welcomeInst, (settings.screenRect.centerx - (settings.instWidth // 2),
                                               settings.screenRect.centery - (settings.instHeight // 2)))
        # flip to foreground
        pygame.display.flip()

        # process continue event
        process_continue_event()
```
In this example we use the `start_welcome_block()` function, because here one can see that attributes of `settings` are used via `settings.______` (e.g. `settings.continueVal != 1`).

## Classes as Independent Entities

So what we have done is separate the settings logic into a settings class that, once instantiated provides us with all necesary settings. Note that this approach is also easier to debug, because we can now create an instance of our class in a test module and test all functionalities. Next, we will abstract even more and create an Experiment and Main class in which even the experiment logic is abstracted.
<br>
<br>
The entire implementation changes that were applied so far can be found in the following python modules:<br><br>
**[config.py](https://github.com/imarevic/psy_python_course/blob/master/notebooks/Chapter10/config.py)**
<br>
**[stroop.py](https://github.com/imarevic/psy_python_course/blob/master/notebooks/Chapter10/stroop.py)**
<br><br>
All other modules remain the same as in the chapter 9 implementation (e.g. TextPresenter.py, directory structure). Take your time and study the settings class implementation carefully and try to understand how the attributes and methods are used in `stroop.py`.